## 2.1 Mini-batch gradient descent
### 2.1.1 Concepts
- Variants of gradient descent
    - **(Batch) gradient descent**: whole training set at each training step. Batch gradient descent is guaranteed to converge to the global minimum for convex cost function and to a local minimum  for non-convex cost function [1].
    - **Mini-batch gradient descent**: `batch_size` samples at each training step. `batch_size` is a **hyper-parameter** (which should be finetuned) and commonly used value is 32, 64, 128, 256, 512.
    - **Stochastic gradient descent**: one sample at each training step. With the help of slowly decreased learning rate, stochastic gradient descent has the same convergence trendency as the batch gradient descent [1].

- Rule of thumb
    - When the size of training set is less than 2000, use batch gradient descent.
    - Otherwise, use mini-batch gradient descent.

### 2.1.2 Mini-batch gradient descent implementation
- Step1: **Shuffle**.
    - Random shuffling is done **synchronously** between X and Y. 
    - `np.random.permutation()`: return the permuted range.
- Step2: **Partition**.
    - Partition the shuffled (X, Y) into mini-batches of size `mini_batch_size`. Note that the size of last mini-batch might be smaller than `mini_batch_size`. 
    - `math.floor()`: round to the nearest integer. `floor(3.9)=3.0`.

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64):
    """
    Creates a list of random minibatches from (X, Y).
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector, of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    # number of training examples
    m = X.shape[1]
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size)
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k*mini_batch_size:(k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k*mini_batch_size:(k+1)*mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches*mini_batch_size:]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches*mini_batch_size:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

## 2.2 Exponentially weighted averages

- $ V_{t} = \beta V_{t-1} + (1 - \beta) \theta_{t}$
- $ V_{t} $ can approximately average $ 1 / (1 - \beta) $ previous $ \theta$s. The larger $ \beta $, the more concentrates on pervious $\theta$s.
- **Bias correction**: $ V_{t} = V_{t} / (1 - \beta^{t})$.
- Exponentially weights: more weights on present $\theta$s and exponentially decayed weights on previous $\theta$s.

## 2.3 Variants of gradient descent optimization algorithm
### 2.3.1 Momentum
- It uses exponentially weighted average of gradients to update parameter. 
- If $\beta = 0$, it is just standard gradient descent without momentum. Common values for $\beta$ range from 0.8 to 0.999 and $\beta = 0.9$ is often a reasonable default. 
- The momentum term **increases updates** for dimensions whose gradients point in the **same directions** and **reduces updates** for dimensions whose gradients **change directions**. As a result, it gains faster convergence and reduced oscillation.

The momentum update rule is, for $l = 1, ..., L$: 

$$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}$$

$$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}$$

where L is the number of layers, $\beta$ is the momentum and $\alpha$ is the learning rate.

### 2.3.2 RMSprop (Root mean square prop)
- It uses exponentially weighted average of the squares of the past gradients to update parameters.

The update rule is, for $l = 1, ..., L$: 

$$\begin{cases}
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
W^{[l]} = W^{[l]} - \alpha \frac{1}{\sqrt{s_{dW^{[l]}}} + \varepsilon}
\end{cases}$$

where:
- L is the number of layers
- $\alpha$ is the learning rate
- $\varepsilon$ (default value is 1e-8) is a very small number to avoid dividing by zero

### 2.3.3 Adam (Adaptive moment estimation)
- It combines an **exponentially weighted average of past gradients with bias correction** and an **exponentially weighted average of the squares of the past gradients with bias correction**. 

The update rule is, for $l = 1, ..., L$: 

$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$
where:
- t counts the number of steps taken of Adam 
- L is the number of layers
- $\beta_1$ (default value is 0.9) and $\beta_2$ (default value is 0.999) are hyperparameters that control the two exponentially weighted averages. [2]
- $\alpha$ is the learning rate
- $\varepsilon$ (default value is 1e-8) is a very small number to avoid dividing by zero

## 2.4 Other concepts 
- **Epoch**: a single pass through the whole training set.
- **Saddle points**
    - Point where one dimension is a local minimum point and another dimension is a local maximum point.
    - They are usually surrounded by a **plateau of zero gradients** in all dimensions.
- **Learning rate decay**
    - $ \alpha = \alpha_0 * 1.0 / (1.0 + decayRate * numEpoch)$
    - Exponential decay: $ \alpha = \alpha_0 * 0.95^{numEpoch}$

References:  
[1] http://ruder.io/optimizing-gradient-descent/index.html